In [3]:
from collections import deque
from queue import PriorityQueue
import heapq
import numpy as np

class Node:
    def __init__(self, value, row, col):
        self.value = value
        self.row = row
        self.col = col
        self.up = None
        self.down = None
        self.left = None
        self.right = None
        self.previousNode = None


class Node_Cost:
    id_counter=1
    adj_list= []

    def __init__(self, value, cost, row, col):
        self.value = value
        self.edgeCost=cost
        self.row = row
        self.col = col
        self.id = Node_Cost.id_counter
        Node_Cost.id_counter += 1
        self.gOfN = None
        self.hOfN = None
        self.fOfN = None
        self.parent = None

    def set_adj_list(self,adjList):
      self.adj_list=adjList

    @classmethod
    def reset_id_counter(cls):
        cls.id_counter = 1

In [4]:
from collections import deque
from queue import PriorityQueue
import heapq
import numpy as np

class Node:
    def __init__(self, value, row, col):
        self.value = value
        self.row = row
        self.col = col
        self.up = None
        self.down = None
        self.left = None
        self.right = None
        self.previousNode = None


class Node_Cost:
    id_counter=1
    adj_list= []

    def __init__(self, value, cost, row, col):
        self.value = value
        self.edgeCost=cost
        self.row = row
        self.col = col
        self.id = Node_Cost.id_counter
        Node_Cost.id_counter += 1
        self.gOfN = None
        self.hOfN = None
        self.fOfN = None
        self.parent = None

    def set_adj_list(self,adjList):
      self.adj_list=adjList

    @classmethod
    def reset_id_counter(cls):
        cls.id_counter = 1

class SearchAlgorithms:
    Path = []
    fullPath = []
    totalCost = -1
    all_nodes=[]
    
    def __init__(self, mazeStr, cost=[]):
        self.maze = [row.split(',') for row in mazeStr.split('. ')]
        self.rows = len(self.maze)
        self.cols = max(len(row) for row in self.maze)
        self.start = None
        self.end = None
        self.graph = self.create_graph()
        self.cost=cost
        self.start2 = None
        self.end2 = None
        if len(cost) == 0:
            self.cost = [1]*(self.rows*self.cols)
        #self.nodes_costs()
        


    def create_graph(self):
        graph = [[None for _ in range(self.cols)] for _ in range(self.rows)]
        for i in range(self.rows):
            for j in range(self.cols):
                if i >= len(self.maze) or j >= len(self.maze[i]):
                    continue
                if self.maze[i][j] == '#':
                    continue
                node = Node(self.maze[i][j], i, j)
                graph[i][j] = node
                if node.value == 'S':
                    self.start = node
                elif node.value == 'E':
                    self.end = node
        for i in range(self.rows):
            for j in range(self.cols):
                if graph[i][j]:
                    if i > 0 and graph[i - 1][j]:
                        graph[i][j].up = graph[i - 1][j]
                    if i < self.rows - 1 and graph[i + 1][j]:
                        graph[i][j].down = graph[i + 1][j]
                    if j > 0 and graph[i][j - 1]:
                        graph[i][j].left = graph[i][j - 1]
                    if j < self.cols - 1 and graph[i][j + 1]:
                        graph[i][j].right = graph[i][j + 1]

        return graph
    
    def BFS(self):
        queue = deque([self.start])
        visited = set()
        fullPath = []
        while queue:
            node = queue.popleft()
            if node.value == 'E':
                fullPath.append(node.row * self.cols + node.col)  # Include the end node in the full path
                path = []
                while node:
                    path.append(node.row * self.cols + node.col)
                    node = node.previousNode
                return path[::-1], fullPath
            if node not in visited and node.value != '#':
                fullPath.append(node.row * self.cols + node.col)
                visited.add(node)
                for neighbor in [node.left, node.down, node.right, node.up]:
                    if neighbor and neighbor not in visited and neighbor.value != '#':
                        neighbor.previousNode = node
                        queue.append(neighbor)

    def DFS(self):
        stack = [self.start]
        visited = set()
        disc = []
        fullPath = []
        while stack:
            node = stack.pop()
            if node.value == 'E':
                fullPath.append((node.row * self.cols) + node.col)  # Include the end node in the full path
                path = []
                while node:
                    path.append((node.row * self.cols) + node.col)
                    node = node.previousNode
                return path[::-1], fullPath
            if node not in visited and node.value != '#':
                fullPath.append((node.row * self.cols) + node.col)
                visited.add(node)
                # for neighbor in [node.up, node.right, node.down, node.left]:
                for neighbor in [node.right, node.left, node.down, node.up]:
                    if neighbor and neighbor not in disc and neighbor not in visited and neighbor.value != '#':
                        neighbor.previousNode = node
                        stack.append(neighbor)
                for neighbor in [node.right, node.left, node.down, node.up]:
                    disc.append(neighbor)  



    def nodes_costs(self):

        #creating nodes and their costs
        for i, row in enumerate(self.maze):
            nodes=[]
            for j, cell in enumerate(row):
                n=None
                idx= i* len(row)+j
                n=Node_Cost(cell, self.cost[idx], i, j)
                
                if n.value == 'S':
                    self.start2 = n
                elif n.value == 'E':
                    self.end2 = n
                
                nodes.append(n)
            self.all_nodes.append(nodes)

        #getting neighbors
        def get_neigh(node,i,j):
            neighbors = []
            directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Right, Left, Down, Up
            directions = directions[::-1]
            for dr, dc in directions:
                new_row = i + dr
                new_col = j + dc
                if (0 <= new_row < len(self.all_nodes) and
                    0 <= new_col < len(self.all_nodes[new_row])):
                    if self.all_nodes[new_row][new_col].value != '#':
                        neighbors.append(self.all_nodes[new_row][new_col].id)
            return neighbors



        for i,l in enumerate(self.all_nodes):
            for j, n in enumerate(l):
                adj_list=[]
                adj_list= get_neigh(n,i,j)
                n.set_adj_list(adj_list)


    def UCS(self):
      self.Path = []
      self.fullPath = []
      self.nodes_costs()
      queue = []
      visited = set()
      heapq.heappush(queue, (0, self.all_nodes[0][0].id, self.all_nodes[0][0], []))  # Initial node with priority 0 and an empty path
      final=[]
      while queue:
          # Get the node with the minimum priority
          current_cost, current_id, current_node, current_path = heapq.heappop(queue)
          self.fullPath.append(current_id)
        #   print("Current node:", current_id-1, "with cost:", current_cost)
          # If it's the goal, return
          if current_node.value == 'E':
              final.append(current_path + [current_node])
              # print(final)
              for i in final:
                for node in i:
                 self.Path.append(node.id-1)
              self.fullPath = [x - 1 for x in self.fullPath]
              print("Cost:", current_cost)
              return self.Path, self.fullPath
          # Mark the current node as visited
          visited.add(current_node)
          # Add all the edges to the priority queue
          for neighbor_id in current_node.adj_list:
              neighbor = self.all_nodes[(neighbor_id - 1) // len(self.all_nodes[0])][(neighbor_id - 1) % len(self.all_nodes[0])]
              if neighbor not in visited:
                  tentative_gOfN = current_cost + neighbor.edgeCost
                  heapq.heappush(queue, (tentative_gOfN, neighbor.id, neighbor, current_path + [current_node]))

    def EcludianHeuristic(self, node: Node_Cost):
        x = (node.row - self.end2.row)**2
        y = (node.col - self.end2.col)**2
        hcost = (x + y)**(1/2)
        return hcost
    
    def ManhattanHeuristic(self, node: Node_Cost):
        x = abs(node.row - self.end2.row)
        y = abs(node.col - self.end2.col)
        hcost = x + y
        return hcost

    def AstarEcludianHeuristic(self):
        open = []
        closed = []
        self.Path = []
        self.fullPath = []
        self.all_nodes.clear()
        Node_Cost.reset_id_counter()
        self.nodes_costs()
        
        self.start2.gOfN = self.start2.edgeCost
        self.start2.hOfN = self.EcludianHeuristic(self.start2)
        self.start2.fOfN = self.start2.gOfN + self.start2.hOfN
        open.append(self.start2)

        while (True):
            min_cost_in_open_list = None

            for node in open:
                if min_cost_in_open_list is None or node.fOfN < min_cost_in_open_list.fOfN:
                    min_cost_in_open_list = node
                elif node.fOfN == min_cost_in_open_list.fOfN:
                    if node.hOfN < min_cost_in_open_list.hOfN:
                        min_cost_in_open_list = node
            open.remove(min_cost_in_open_list)
            closed.append(min_cost_in_open_list)
            if min_cost_in_open_list.value == 'E':
                break

            #neighbors = [node for node in self.all_nodes if node[0] in min_cost_in_open_list.adj_list]
            for neighbor_id in min_cost_in_open_list.adj_list:
                neighbor = self.all_nodes[(neighbor_id - 1) // len(self.all_nodes[0])][(neighbor_id - 1) % len(self.all_nodes[0])]
                if not (neighbor in closed):
                    gcost = neighbor.edgeCost + min_cost_in_open_list.edgeCost
                    if neighbor.gOfN is None or gcost < neighbor.gOfN: 
                        neighbor.parent = min_cost_in_open_list
                        neighbor.gOfN = neighbor.edgeCost + neighbor.parent.gOfN
                        neighbor.hOfN = self.EcludianHeuristic(neighbor)
                        neighbor.fOfN = neighbor.gOfN + neighbor.hOfN
                if not (neighbor in open) and not (neighbor in closed):
                    open.append(neighbor)

        path_node = closed[-1]
        while(True):
            if path_node.parent is not None:
                self.Path.append(path_node.id-1)
                path_node = path_node.parent
                self.totalCost += path_node.edgeCost
            else:
                break
        
        self.Path.append(0)
        self.Path = self.Path[::-1]
        
        for node in closed:
            self.fullPath.append(node.id-1)

        return self.Path, self.fullPath, self.totalCost +1

    def AstarManhattanHeuristic(self):
        open = []
        closed = []
        self.Path = []
        self.fullPath = []
        self.all_nodes.clear()
        Node_Cost.reset_id_counter()
        self.nodes_costs()
        
        self.start2.gOfN = self.start2.edgeCost
        self.start2.hOfN = self.ManhattanHeuristic(self.start2)
        self.start2.fOfN = self.start2.gOfN + self.start2.hOfN
        open.append(self.start2)
        #self.fullPath.append(self.start2.id-1)

        while (True):
            min_cost_in_open_list = None

            for node in open:
                if min_cost_in_open_list is None or node.fOfN < min_cost_in_open_list.fOfN:
                    min_cost_in_open_list = node
                #elif node.fOfN == min_cost_in_open_list.fOfN:
                #    if node.hOfN < min_cost_in_open_list.hOfN:
                #        min_cost_in_open_list = node
            open.remove(min_cost_in_open_list)
            closed.append(min_cost_in_open_list)
            if min_cost_in_open_list.value == 'E':
                break

            for neighbor_id in min_cost_in_open_list.adj_list:
                neighbor = self.all_nodes[(neighbor_id - 1) // len(self.all_nodes[0])][(neighbor_id - 1) % len(self.all_nodes[0])]
                if not (neighbor in closed):
                    gcost = neighbor.edgeCost + min_cost_in_open_list.edgeCost
                    if neighbor.gOfN is None or gcost < neighbor.gOfN: 
                        neighbor.parent = min_cost_in_open_list
                        neighbor.gOfN = neighbor.edgeCost + neighbor.parent.gOfN
                        neighbor.hOfN = self.ManhattanHeuristic(neighbor)
                        neighbor.fOfN = neighbor.gOfN + neighbor.hOfN
                if not (neighbor in open) and not (neighbor in closed) and neighbor.value != '#':
                    open.append(neighbor)

        path_node = closed[-1]
        while(True):
            if path_node.parent is not None:
                self.Path.append(path_node.id-1)
                path_node = path_node.parent
            else:
                break
        
        self.Path.append(0)
        self.Path = self.Path[::-1]
        
        for node in closed:
            self.fullPath.append(node.id-1)

        return self.Path, self.fullPath, len(self.Path)-1


def main():
    s1 = SearchAlgorithms('S,.,.,#,.,.,. .,#,.,.,.,#,. .,#,.,.,.,.,. .,.,#,#,.,.,. #,.,#,E,.,#,.')
    path, fullPath = s1.BFS()
    print('BFS Path: ' + str(path), end='\nFull Path is: ')
    print(fullPath)


    s2 = SearchAlgorithms('S,.,.,#,.,.,. .,#,.,.,.,#,. .,#,.,.,.,.,. .,.,#,#,.,.,. #,.,#,E,.,#,.')

    path, fullPath = s2.DFS()
    print('DFS Path: ' + str(path), end='\nFull Path is: ')
    print(fullPath)


    s3 = SearchAlgorithms('S,.,.,#,.,.,. .,#,.,.,.,#,. .,#,.,.,.,.,. .,.,#,#,.,.,. #,.,#,E,.,#,.' , [0, 15, 2, 100, 60, 35, 30, 3
                            , 100, 2, 15, 60, 100, 30, 2
                            , 100, 2, 2, 2, 40, 30, 2, 2
                            , 100, 100, 3, 15, 30, 100, 2
                            , 100, 0, 2, 100, 30])
    path, fullPath = s3.UCS()
    print('UCS Path: ' + str(path), end='\nFull Path is: ')
    print(fullPath)


    s4 = SearchAlgorithms('S,.,.,#,.,.,. .,#,.,.,.,#,. .,#,.,.,.,.,. .,.,#,#,.,.,. #,.,#,E,.,#,.',
                          [0, 15, 2, 100, 60, 35, 30, 3
                              , 100, 2, 15, 60, 100, 30, 2
                              , 100, 2, 2, 2, 40, 30, 2, 2
                              , 100, 100, 3, 15, 30, 100, 2
                              , 100, 0, 2, 100, 30])
    path, fullPath, cost = s4.AstarEcludianHeuristic()
    print('AstarEcludianHeuristic Path: ' + str(path), end='\nFull Path is: ')
    print(fullPath)
    print(f'Cost: {cost}')

    s5 = SearchAlgorithms('S,.,.,#,.,.,. .,#,.,.,.,#,. .,#,.,.,.,.,. .,.,#,#,.,.,. #,.,#,E,.,#,.')
    path, fullPath, cost = s5.AstarManhattanHeuristic()
    print('AstarManhattanHeuristic Path: ' + str(path), end='\nFull Path is: ')
    print(fullPath)
    print(f'Cost: {cost}')




main()


BFS Path: [0, 1, 2, 9, 10, 11, 18, 25, 32, 31]
Full Path is: [0, 7, 1, 14, 2, 21, 9, 22, 16, 10, 29, 17, 11, 18, 4, 25, 19, 5, 32, 26, 20, 6, 31]
DFS Path: [0, 1, 2, 9, 16, 17, 18, 25, 32, 31]
Full Path is: [0, 7, 14, 21, 22, 29, 1, 2, 9, 16, 17, 18, 11, 4, 5, 6, 13, 20, 27, 34, 26, 25, 32, 31]
Cost: 30
UCS Path: [0, 1, 2, 9, 16, 17, 18, 25, 32, 31]
Full Path is: [0, 7, 14, 21, 22, 29, 1, 2, 9, 16, 17, 18, 25, 32, 31]
AstarEcludianHeuristic Path: [0, 1, 2, 9, 16, 17, 18, 25, 32, 31]
Full Path is: [0, 7, 14, 21, 22, 29, 1, 2, 9, 16, 17, 18, 25, 32, 31]
Cost: 30
AstarManhattanHeuristic Path: [0, 1, 2, 9, 10, 11, 18, 25, 32, 31]
Full Path is: [0, 7, 1, 14, 2, 21, 9, 22, 16, 10, 29, 17, 11, 18, 25, 32, 31]
Cost: 9
